In [1]:
import pandas as pd
import numpy as np
import sncosmo
from sncosmo.salt2utils import BicubicInterpolator
import glob
from scipy.interpolate import (
    InterpolatedUnivariateSpline as Spline1d,
    RectBivariateSpline as Spline2d
)
import math
import os
from tabulate import tabulate

In [2]:
#from scipy.interpolate import RectBivariateSpline

# A new class for sncosmo
class EXP_LC(sncosmo.Source):
    
    _param_names = ['x0', 'x1', 'x2','x3','x4','x5','x6','x7','x8','x9', 'c']
    param_names_latex = ['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9', 'c']
    
    
    def __init__(self, phase, wave, flux0, flux1, flux2, flux3, flux4, flux5, flux6, flux7, flux8, flux9, LC, name='EXP', version='2021'):

        self.name = name
        self.version = version
        self._phase = phase
        self._wave = wave
        
        self._model_flux0  = BicubicInterpolator(phase, wave, flux0.T)
        self._model_flux1  = BicubicInterpolator(phase, wave, flux1.T)
        self._model_flux2  = BicubicInterpolator(phase, wave, flux2.T)
        self._model_flux3  = BicubicInterpolator(phase, wave, flux3.T)
        self._model_flux4  = BicubicInterpolator(phase, wave, flux4.T)
        self._model_flux5  = BicubicInterpolator(phase, wave, flux5.T)
        self._model_flux6  = BicubicInterpolator(phase, wave, flux6.T)
        self._model_flux7  = BicubicInterpolator(phase, wave, flux7.T)
        self._model_flux8  = BicubicInterpolator(phase, wave, flux8.T)
        self._model_flux9  = BicubicInterpolator(phase, wave, flux9.T)
        self._model_ext =  Spline1d(LC['wave'], LC['ext'],  k=1)  


        self._parameters = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])  # initial guess
        

    def _flux(self, phase, wave):
        
        x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, c = self._parameters
        #print('flux!')
        return x0 * (self._model_flux0(phase, wave) +  x1 * self._model_flux1(phase, wave) +  x2 * self._model_flux2(phase, wave) +  x3 * self._model_flux3(phase, wave) +  x4 * self._model_flux4(phase, wave)  +  x5 * self._model_flux5(phase, wave) +  x6 * self._model_flux6(phase, wave) +  x7 * self._model_flux7(phase, wave) +  x8 * self._model_flux8(phase, wave) +  x9 * self._model_flux9(phase, wave))* 10.** (-0.4 * self._model_ext(wave) * c)

In [3]:
files=glob.glob("./JLA_LCS/*.list")
print("JLA files in path: ", len(files))

JLA files in path:  740


## Factor analysis templates

In [4]:
M0 = pd.read_csv("./Templates/fa/M0.dat", names=["time", "wave", "flux"], sep="\s+")
M1 = pd.read_csv("./Templates/fa/M1.dat", names=["time", "wave", "flux"], sep="\s+")
M2 = pd.read_csv("./Templates/fa/M2.dat", names=["time", "wave", "flux"], sep="\s+")
M3 = pd.read_csv("./Templates/fa/M3.dat", names=["time", "wave", "flux"], sep="\s+")
M4 = pd.read_csv("./Templates/fa/M4.dat", names=["time", "wave", "flux"], sep="\s+")
M5 = pd.read_csv("./Templates/fa/M5.dat", names=["time", "wave", "flux"], sep="\s+")
M6 = pd.read_csv("./Templates/fa/M6.dat", names=["time", "wave", "flux"], sep="\s+")
M7 = pd.read_csv("./Templates/fa/M7.dat", names=["time", "wave", "flux"], sep="\s+")
M8 = pd.read_csv("./Templates/fa/M8.dat", names=["time", "wave", "flux"], sep="\s+")
M9 = pd.read_csv("./Templates/fa/M9.dat", names=["time", "wave", "flux"], sep="\s+")

In [5]:
M0.head()

,time,wave,flux
0,-10,3500,0.146131
1,-9,3500,0.155753
2,-8,3500,0.163890
3,-7,3500,0.170134
4,-6,3500,0.174158


In [6]:
SCALE_FACTOR=10**(-12)

M0_reshape = np.array(M0["flux"]).reshape(501, 61)*SCALE_FACTOR
M1_reshape = np.array(M1["flux"]).reshape(501, 61)*SCALE_FACTOR
M2_reshape = np.array(M2["flux"]).reshape(501, 61)*SCALE_FACTOR
M3_reshape = np.array(M3["flux"]).reshape(501, 61)*SCALE_FACTOR
M4_reshape = np.array(M4["flux"]).reshape(501, 61)*SCALE_FACTOR
M5_reshape = np.array(M5["flux"]).reshape(501, 61)*SCALE_FACTOR
M6_reshape = np.array(M6["flux"]).reshape(501, 61)*SCALE_FACTOR
M7_reshape = np.array(M7["flux"]).reshape(501, 61)*SCALE_FACTOR
M8_reshape = np.array(M8["flux"]).reshape(501, 61)*SCALE_FACTOR
M9_reshape = np.array(M9["flux"]).reshape(501, 61)*SCALE_FACTOR

## SALT2 templates

In [7]:
M0_SALT2 = pd.read_csv("./Templates/salt2_betoule_et_al/salt2_template_0.dat", names=["time", "wave", "flux"], sep="\s+")
M1_SALT2 = pd.read_csv("./Templates/salt2_betoule_et_al/salt2_template_1.dat", names=["time", "wave", "flux"], sep="\s+")

In [8]:
c = pd.read_csv("./Templates/salt2_colour_law_guy2010.dat", header = None, sep = "\s+") # color term
c.columns = ["wave", "ext"]

In [9]:
M0_SALT2.head()

,time,wave,flux
0,-20.0,2000.0,0.0
1,-20.0,2010.0,0.0
2,-20.0,2020.0,0.0
3,-20.0,2030.0,0.0
4,-20.0,2040.0,0.0


In [10]:
M0_SALT2_reshape = np.array(M0_SALT2["time"]).reshape(721, 71)
M1_SALT2_reshape = np.array(M1_SALT2["flux"]).reshape(721, 71)

In [11]:
M0_SALT2_reshape

array([[-20., -20., -20., ..., -20., -20., -20.],
       [-20., -20., -20., ..., -20., -20., -20.],
       [-20., -20., -20., ..., -20., -20., -20.],
       ...,
       [ 50.,  50.,  50., ...,  50.,  50.,  50.],
       [ 50.,  50.,  50., ...,  50.,  50.,  50.],
       [ 50.,  50.,  50., ...,  50.,  50.,  50.]])

In [12]:
gridx = np.linspace(-10, 50, 61)
gridy = np.linspace(3500, 8500, 501)

exp_lc = EXP_LC(gridx, gridy, M0_reshape, M1_reshape, M2_reshape, M3_reshape, M4_reshape, M5_reshape, M6_reshape, M7_reshape, M8_reshape, M9_reshape, c)

In [13]:
table = []

In [14]:
exc = [50, 158, 211, 339, 409, 592]

In [15]:
for i in range(0, len(files)):
    print(i)
    if i not in exc:
        data = sncosmo.read_lc(str(files[i]), format='salt2')

        if data["Filter"][0][:-3] == "MEGACAMPSF":

            r = math.sqrt(data.meta['X_FOCAL_PLANE']**2 + data.meta['Y_FOCAL_PLANE']**2)


            megacg = sncosmo.get_bandpass('megacampsf::g', r)
            megacr = sncosmo.get_bandpass('megacampsf::r', r)
            megaci = sncosmo.get_bandpass('megacampsf::i', r)
            megacz = sncosmo.get_bandpass('megacampsf::z', r)

            sncosmo.register(megacg, 'MEGACAMPSF::g', force = True)
            sncosmo.register(megacr, 'MEGACAMPSF::r', force = True)
            sncosmo.register(megaci, 'MEGACAMPSF::i', force = True)
            sncosmo.register(megacz, 'MEGACAMPSF::z', force = True)

        rname = data.meta['SN']
        # original salt2
        model_salt2 = sncosmo.Model(source="salt2",
                                    effects=[sncosmo.F99Dust()],
                                    effect_names=['mw'],
                                    effect_frames=['obs'])

        model_salt2.set(mwebv=data.meta['MWEBV'],
                        z = data.meta['Z_HELIO'])

        model_3 = sncosmo.Model(source=exp_lc,
                                effects=[sncosmo.F99Dust()],
                                effect_names=['mw'],
                                effect_frames=['obs'])

        model_3.set(mwebv=data.meta['MWEBV'],
                    z = data.meta['Z_HELIO'],
                    x3 = 0,
                    x4 = 0,
                    x5 = 0,
                    x6 = 0,
                    x7 = 0,
                    x8 = 0,
                    x9 = 0,
                    c = 0)

        model_2_lc = sncosmo.Model(source=exp_lc,
                                   effects=[sncosmo.F99Dust()],
                                   effect_names=['mw'],
                                   effect_frames=['obs'])

        model_2_lc.set(mwebv=data.meta['MWEBV'],
                       z = data.meta['Z_HELIO'],
                       x2 = 0,
                       x3 = 0,
                       x4 = 0,
                       x5 = 0,
                       x6 = 0,
                       x7 = 0,
                       x8 = 0,
                       x9 = 0)

        result_salt2, m_salt2 = sncosmo.fit_lc(data, model_salt2, ['t0', 'x0', 'x1', 'c'], modelcov=False, phase_range=(-10, 45.), wave_range=(4000., 8000.), verbose=False)

        result_3, m_3 = sncosmo.fit_lc(data, model_3, ['t0', 'x0', 'x1', 'x2'], modelcov=False, phase_range=(-10, 45.), wave_range=(4000., 8000.), verbose=False)

        result_2, m_2 = sncosmo.fit_lc(data, model_2_lc, ['t0', 'x0', 'x1', 'c'], modelcov=False, phase_range=(-10, 45.), wave_range=(4000., 8000.), verbose=False)



        if result_salt2.success == True:
            if result_salt2.ndof > 2:
                print(result_salt2.chisq/result_salt2.ndof)
                table.append((rname, result_salt2.parameters[0], result_salt2.parameters[1], result_salt2.errors['t0'], result_salt2.parameters[2], result_salt2.errors['x0'], result_salt2.parameters[3], result_salt2.errors['x1'], result_salt2.parameters[4], result_salt2.errors['c'], data.meta['MWEBV'], result_salt2.covariance[1][2], result_salt2.covariance[1][3], result_salt2.covariance[2][3], result_salt2.chisq/result_salt2.ndof, "salt2"))
        if result_3.success == True:
            if result_3.ndof > 2:
                print(result_3.chisq/result_3.ndof)
                table.append((rname, result_3.parameters[0], result_3.parameters[1], result_3.errors['t0'], result_3.parameters[2], result_3.errors['x0'], result_3.parameters[3], result_3.errors['x1'], result_3.parameters[4], result_3.errors['x2'], data.meta['MWEBV'], result_3.covariance[1][2], result_3.covariance[1][3], result_3.covariance[2][3], result_3.chisq/result_3.ndof, "exp3"))
        if result_2.success == True:
            if result_2.ndof > 2:    
                print(result_2.chisq/result_2.ndof)
                table.append((rname, result_2.parameters[0], result_2.parameters[1], result_2.errors['t0'], result_2.parameters[2], result_2.errors['x0'], result_2.parameters[3], result_2.errors['x1'], result_2.parameters[12], result_2.errors['c'], data.meta['MWEBV'], result_2.covariance[1][2], result_2.covariance[1][3], result_2.covariance[2][3], result_2.chisq/result_2.ndof, "exp2lc"))


0


/home/joao/.local/lib/python3.8/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: megacampsf::r at 17.085289, megacampsf::g at 17.085289(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +


0.13693331256322053
0.09212407429525285
0.15429154140231846
1


/home/joao/.local/lib/python3.8/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: megacampsf::g at 14.933665(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +


1.0320323419747073
11.012677736710204
5.970976792636613
2


...

1.4313811054772054
1.9206233899148772
1.841222726098055
738


/home/joao/.local/lib/python3.8/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: megacampsf::g at 16.806184, megacampsf::r at 16.806184(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +


0.9725056912728909
1.1546056481432514
1.916372304970279
739


/home/joao/.local/lib/python3.8/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: sdss::g, sdss::u, sdss::z(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +


0.687776553145968
1.0717476280749418
0.9580034171338113


In [16]:
len(table)

2125

In [17]:
save_path = './'

In [18]:
completeName = os.path.join(save_path,"ajuste_lc.txt")      

header = "name     redshift     t0      t0err          x0         x0err       x1          x1err          x2     x2err   mwebv      cov_x0x1       cov_x0c       cov_x1x2  chiquad/dof modelo"

f = open(completeName, 'w')               
f.write(header + "\n")
f.write(tabulate(table, tablefmt="plain"))
f.close()